In [ ]:
import os
import glob
CAPTCHA_IMAGE_FOLDER = "captcha_images"
OUTPUT_FOLDER = "extracted_letter_images"
# ambil semua file yang ada di folder *
captcha_image_files = glob.glob(os.path.join(CAPTCHA_IMAGE_FOLDER, "*"))

In [ ]:
counts = {}

In [ ]:
def preprocessImage(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #bikin jadi grayscale
    #nambahin pixel sehingga fix 8 8 dengan nilai pixel sesuai dengan ujung gambar asli
    gray = cv2.copyMakeBorder(gray, 8, 8, 8, 8, cv2.BORDER_REPLICATE) 
    #threshold untuk pixel, yg jadi 0 cv2.THRESH_BINARY_INV, 255 pake cv2.THRESH_OTSU
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    return gray, thresh

def contoursToRectangles(contours):
    letter_image_regions = []
    for contour in contours:
        #bikin boundary kotak per hurufnya
        (x, y, w, h) = cv2.boundingRect(contour)
        if w / h > 1.25:
            half_width = int(w / 2)
            letter_image_regions.append((x, y, half_width, h))
            letter_image_regions.append((x + half_width, y, half_width, h))
        else:
            letter_image_regions.append((x, y, w, h))
    return letter_image_regions

In [ ]:
import cv2
import imutils
import numpy as np

for (i, captcha_image_file) in enumerate(captcha_image_files):
    print("Processing image {}/{}".format(i + 1, len(captcha_image_files)))
    filename = os.path.basename(captcha_image_file)
    captcha_correct_text = os.path.splitext(filename)[0]#ngambil nilai angka dari captcha

    image = cv2.imread(captcha_image_file)
    gray, preprocessedImage = preprocessImage(image) #gambar & threshold
    #3 argumen, imagenya, retrieval metodenya sama aproksimasi countour methodnya
    contours = cv2.findContours(preprocessedImage.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #output counter itu koordinat boundarynya
    contours = contours[0]

    letter_image_regions = contoursToRectangles(contours)

    if len(letter_image_regions) != 4:
        continue
    #sort sesuai urutann hurufnya (countour[0])
    letter_image_regions = sorted(letter_image_regions, key=lambda x: x[0])

    for letter_bounding_box, letter_text in zip(letter_image_regions, captcha_correct_text):
        #zip gabung 2 list jadi 1 tuple gitu
        x, y, w, h = letter_bounding_box
        letter_image = gray[y - 2:y + h + 2, x - 2:x + w + 2]
        save_path = os.path.join(OUTPUT_FOLDER, letter_text)
        if not os.path.exists(save_path):
            os.makedirs(save_path)

        count = counts.get(letter_text, 1)
        p = os.path.join(save_path, "{}.png".format(str(count).zfill(6)))
        cv2.imwrite(p, letter_image)

        counts[letter_text] = count + 1